# Приложение Flask для запроса ChatGPT

Flask application for ChatGPT request



Приглашаю в Телеграм общаться по это теме: https://t.me/AiExp01

In [1]:
#@title Сохраняйте эту ячейку активной, чтобы Colab не отключил вас. { display-mode: "form" }

#@markdown Нажмите кнопку воспроизведения на музыкальном проигрывателе, который появится ниже:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
!pip -q install pyngrok
!pip -q install openai
!pip -q install loguru
!pip -q install tiktoken
!pip -q install langchain
!pip -q install langchain-openai
# !pip -q install faiss-cpu
!pip -q install faiss-gpu
!pip -q install langchain-community
!git clone https://github.com/kvoloshenko/GPT_RAG_TOOLS_01.git
%cd /content/GPT_RAG_TOOLS_01/Python
# from tools_01 import get_google_url
import tools_01 as tls
import db_tools_01 as dbt
import gpt_tools_01 as gpt

In [ ]:
# Подключаем Google disk
# для этого необходима учетная запись на гугл
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from flask import Flask, request, json
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from google.colab import userdata
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
import os
from loguru import logger

DB_DIR_NAME = '/content/drive/MyDrive/_AI/Db/'
CHUNK_SIZE = 1024
NUMBER_RELEVANT_CHUNKS = 5
TEMPERATURE = 0.8            # Температура модели
LL_MODEL = "gpt-3.5-turbo-0613" # Модель

logger.add("/content/drive/MyDrive/_AI/Logs/bot_debug.log", format="{time} {level} {message}", level="DEBUG", rotation="100 KB", compression="zip")


# Установка Ngrok_API_KEY
Ngrok_API_KEY=userdata.get('Ngrok_API_KEY')
ngrok.set_auth_token(Ngrok_API_KEY)

# Установка OpenAI API key
OpenAI_API_KEY=userdata.get('OpenAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OpenAI_API_KEY

# LL_MODEL = "gpt-4-0613"
LL_MODEL = "gpt-3.5-turbo"
print(f'LL_MODEL = {LL_MODEL}')
client = OpenAI(
    api_key=OpenAI_API_KEY
)
port_no = 5000
app = Flask(__name__)

public_url =  ngrok.connect(port_no).public_url
print(f"To acces the Gloable link please click {public_url}")
tls.write_to_file(public_url, "/content/drive/MyDrive/_AI/Db/public_url.txt")

In [9]:
# Функции для Flask
UPLOAD_FOLDER = '/content/sample_data'
ALLOWED_EXTENSIONS = {'wav'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

@app.route('/chat', methods=['POST'])
def chat():
    data = json.loads(request.data)
    print(type(data), f'chat data={data}')
    system_content = data['system_content']
    user_content = data['user_content']
    completion = gpt.get_simple_answer_gpt(system_content, user_content, LL_MODEL, TEMPERATURE, client)
    answer = completion.choices[0].message.content
    data['answer'] = answer
    print(f'return data={data}')
    return data

@app.route('/answer_gpt', methods=['POST'])
def answer_gpt():
    data = json.loads(request.data)
    print(type(data), f'chat data={data}')
    system_content = data['system_content']
    user_content = data['user_content']
    ba = data['ba']
    answer = gpt.get_answer_gpt(system_content, user_content, ba, LL_MODEL, TEMPERATURE, NUMBER_RELEVANT_CHUNKS, DB_DIR_NAME, client)
    data['answer'] = answer
    print(f'return data={data}')
    return data


@app.route('/transcript', methods=['POST'])
# https://flask.palletsprojects.com/en/2.3.x/patterns/fileuploads/
def transcript():
    logger.debug("transcript")

    # check if the post request has the file part
    if 'file' not in request.files:
        flash('No file part')
        return 'No file part'
    file = request.files['file']
    logger.debug(type(file))
    args = request.args
    language = request.args.get('language')
    # If the user does not select a file, the browser submits an
    # empty file without a filename.
    if file.filename == '':
        flash('No selected file')
        return 'No file part'
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    my_file_1 = UPLOAD_FOLDER + "/"+ filename
    transcript= gpt.get_transcript(client, my_file_1, language=language )
    data = {"transcript": transcript}
    return data

@app.route('/db_create', methods=['POST'])
def db_create():
    data = json.loads(request.data)
    print(f'db_create data={data}')
    knowledge_base_url = data['knowledge_base_url']
    ba = data['ba']
    BA = ba
    KNOWLEDGE_BASE_URL = tls.get_google_url(knowledge_base_url)
    print(f'KNOWLEDGE_BASE_URL={KNOWLEDGE_BASE_URL}')
    embeddings = OpenAIEmbeddings()
    knowledge_base_text = tls.load_document_text(KNOWLEDGE_BASE_URL)
    db, ar_db_file_name, ar_chunk_num = dbt.create_db(ba, knowledge_base_text, DB_DIR_NAME, CHUNK_SIZE, embeddings)
    return data

In [ ]:
app.run(port=port_no)

# Вопросы и пожелания сюда:
 https://t.me/AiExp01